In [1]:
from src.controller.HwpController import HwpController
from collections import OrderedDict

import re, uuid
import numpy as np

In [2]:
controller = HwpController()
path = '/Users/dobby/Desktop/Workspace/3월 계측보고서(신안산선2공구A-1).hwp'

controller.get_table_list(path, '일자별 발파 및 계측 현황')

unmatched field end


In [3]:
import re, uuid
import numpy as np

from src.service.ParseService import ParseService


class ComplicatedParser:
    def delete_non_target_data(self, table_data):
        """
        한글 표에서 원하지 않은 부분까지 나온 데이터를 정리하여 리스트로 반환합니다.
        target_data_text에 표에 반복적으로 들어가는 텍스트를 입력하여 필요없는 데이터를 삭제합니다.
        """
        target_data_text = ['일시', '발파', '진동', '소음', 'STA', '시간']

        target_data = [
            sublist for sublist in table_data
            if any(
                entry['row'] in ['0', '1'] and any(keyword in entry['text'] for keyword in target_data_text)
                for entry in sublist
            )
        ]
        
        target_data = [[item for item in items if item['text'] != ''] for items in target_data]

        return target_data

    def extract_columns(self, table_list):
        """
        한글 표에서 공통적인 컬럼 부분을 추출합니다.
        컬럼은 대부분 표의 시작 부분에 작성되기 때문에 row 값은 0 혹은 1에 위치하게 됩니다.
        이후 컬럼 값들이 딕셔너리로 저장되어 있기 때문에 중복을 제거 후 리스트로 반환합니다.
        """
        columns = []
        for items in table_list:
            for item in items:
                if item not in [i for i in columns]:
                    if int(item['row']) == 0 or int(item['row']) == 1:
                        columns.append(item)

        return columns
    
    def extract_non_column_data(self, table_list, columns):
        """
        한글 표에서 컬럼 부분을 제거한 나머지 데이터들을 반환합니다.
        """
        return [[item for item in items[len(columns):] if not int(item['colspan']) > 1]for items in table_list]


    def group_by_date(self, dict_list):
        """
        한글 표 데이터를 날짜 별로 분류하여 리스트로 저장하여 반환합니다.
        날짜 별 분류는 하나의 TableCell에 포함된 동일한 row값들 끼리 묶는 것으로 수행합니다.
        """
        group_list = []

        for items in dict_list:
            rows = list(set([int(item['row']) for item in items]))
            for row in rows:
                temp = [item for item in items if int(item['row'])==row]
                group_list.append(temp)
        
        return [items for items in group_list if len(items) > 1] 
    
    def update_merge_data(self, group_list):
        """
        한글 표에 병합 처리된 셀에 대한 데이터 처리를 완료한 뒤 리스트로 반환합니다.
        SimpleParser에서는 병합 셀이 아닌 셀의 최초 값이 공통 값으로 결정되기 때문에 이를 통해 값을 추가합니다.
        """
        cached_merge_data = []
        cached_head_data = []
        
        for idx, items in enumerate(group_list):
            head_data = [data for data in items if int(data['col']) == 0 and int(data['rowspan']) > 1]
            merge_data = [data for data in items if int(data['col']) != 0 and int(data['rowspan']) > 1]
            
            if head_data:
                cached_head_data = head_data
            if merge_data:
                cached_merge_data = merge_data

            new_items = []
            if any(data not in items for data in cached_head_data):
                new_items.extend(cached_head_data)
            if any(data not in items for data in cached_merge_data):
                new_items.extend(cached_merge_data)

            group_list[idx] = new_items + items
        
        return group_list
    
    def serialize_to_dict(self, group_list, columns):
        """
        컬럼 리스트와 파싱이 끝난 그룹 리스트를 이용해서 데이터를 분류한 뒤 리스트로 반환합니다.
        컬럼 리스트에 대응하는 값들을 그룹 리스트에서 찾아서 추가해주는 작업을 수행합니다.
        """
        serialize_list = []
        find_word = ['일시', '시간', 'cm', 'dB', '측정위치']

        columns = [item for item in columns if int(item['colspan']) <= 1 and any(word in item['text'] for word in find_word)]
        for items in group_list:
            data = {column['text']: item['text'] for column, item in zip(columns, items)}
            serialize_list.append(data)

        return serialize_list


In [4]:
service = ComplicatedParser()

In [5]:
table_list = service.delete_non_target_data(controller.get_list())

In [6]:
columns = service.extract_columns(table_list)

In [7]:
dict_list = service.extract_non_column_data(table_list, columns)

In [23]:
group_list = service.group_by_date(dict_list)
group_list

[[{'row': '2', 'col': '0', 'colspan': '1', 'rowspan': '4', 'text': '3월1일'},
  {'row': '2', 'col': '1', 'colspan': '1', 'rowspan': '1', 'text': '1회'},
  {'row': '2', 'col': '2', 'colspan': '1', 'rowspan': '1', 'text': '7:01'},
  {'row': '2', 'col': '3', 'colspan': '1', 'rowspan': '1', 'text': '1.6-2.0'},
  {'row': '2', 'col': '4', 'colspan': '1', 'rowspan': '1', 'text': '300.0'},
  {'row': '2',
   'col': '5',
   'colspan': '1',
   'rowspan': '1',
   'text': '7k+771.25~774.25(종점)'},
  {'row': '2', 'col': '6', 'colspan': '1', 'rowspan': '1', 'text': 'N/T'},
  {'row': '2', 'col': '7', 'colspan': '1', 'rowspan': '1', 'text': 'N/T'},
  {'row': '2', 'col': '8', 'colspan': '1', 'rowspan': '1', 'text': 'N/T'},
  {'row': '2', 'col': '9', 'colspan': '1', 'rowspan': '1', 'text': '국원물산'}],
 [{'row': '3', 'col': '1', 'colspan': '1', 'rowspan': '2', 'text': '2회'},
  {'row': '3', 'col': '2', 'colspan': '1', 'rowspan': '2', 'text': '11:47'},
  {'row': '3', 'col': '3', 'colspan': '1', 'rowspan': '2', 't

In [24]:
# cached_merge_data = []
# cached_head_data = []

# for idx, items in enumerate(group_list):
#     head_data = [data for data in items if int(data['col']) == 0 and int(data['rowspan']) > 1]
#     merge_data = [data for data in items if int(data['col']) != 0 and int(data['rowspan']) > 1]
    
#     # head_data는 변경되는데 merge_data가 rowspan조건에 부합하지 않아서 이전의 merge_data값을 가지고 있어서 에러가 생김
#     if head_data:
#         if len(merge_data) == 0 and cached_merge_data:
#             cached_merge_data = []
#         cached_head_data = head_data
#     if merge_data:
#         cached_merge_data = merge_data

#     for item in items:
#         print(item)
#     print(f'head: {head_data}')
#     print(f'cHead: {cached_head_data}')

#     print(f'merge: {merge_data}')
#     print(f'cMerge: {cached_merge_data}')
#     print("\n\n")

#     new_items = []
#     if any(data not in items for data in cached_head_data):
#         new_items.extend(cached_head_data)
#     if any(data not in items for data in cached_merge_data):
#         new_items.extend(cached_merge_data)

#     group_list[idx] = new_items + items
#     print('=== RESULT ===')
#     for i in group_list[idx]:
#         print(i)
#     print("\n\n")

In [45]:
data = [
    [{'row': '2', 'col': '0', 'colspan': '1', 'rowspan': '4', 'text': '3월1일'},
     {'row': '2', 'col': '1', 'colspan': '1', 'rowspan': '1', 'text': '1회'},
     {'row': '2', 'col': '2', 'colspan': '1', 'rowspan': '1', 'text': '7:01'},
     {'row': '2', 'col': '3', 'colspan': '1', 'rowspan': '1', 'text': '1.6-2.0'},
     {'row': '2', 'col': '4', 'colspan': '1', 'rowspan': '1', 'text': '300.0'},
     {'row': '2', 'col': '5', 'colspan': '1', 'rowspan': '1', 'text': '7k+771.25~774.25(종점)'},
     {'row': '2', 'col': '6', 'colspan': '1', 'rowspan': '1', 'text': 'N/T'},
     {'row': '2', 'col': '7', 'colspan': '1', 'rowspan': '1', 'text': 'N/T'},
     {'row': '2', 'col': '8', 'colspan': '1', 'rowspan': '1', 'text': 'N/T'},
     {'row': '2', 'col': '9', 'colspan': '1', 'rowspan': '1', 'text': '국원물산'}],
    [{'row': '3', 'col': '1', 'colspan': '1', 'rowspan': '2', 'text': '2회'},
     {'row': '3', 'col': '2', 'colspan': '1', 'rowspan': '2', 'text': '11:47'},
     {'row': '3', 'col': '3', 'colspan': '1', 'rowspan': '2', 'text': '1.6-2.0'},
     {'row': '3', 'col': '4', 'colspan': '1', 'rowspan': '2', 'text': '95.7'},
     {'row': '3', 'col': '5', 'colspan': '1', 'rowspan': '2', 'text': '5k+172.90~171.65(시점)'},
     {'row': '3', 'col': '6', 'colspan': '1', 'rowspan': '1', 'text': '0.209'},
     {'row': '3', 'col': '7', 'colspan': '1', 'rowspan': '1', 'text': '72.51'},
     {'row': '3', 'col': '8', 'colspan': '1', 'rowspan': '1', 'text': '73.35'},
     {'row': '3', 'col': '9', 'colspan': '1', 'rowspan': '1', 'text': '풍전빌딩'}],
    [{'row': '4', 'col': '6', 'colspan': '1', 'rowspan': '1', 'text': '0.141'},
     {'row': '4', 'col': '7', 'colspan': '1', 'rowspan': '1', 'text': '71.42'},
     {'row': '4', 'col': '8', 'colspan': '1', 'rowspan': '1', 'text': '74.79'},
     {'row': '4', 'col': '9', 'colspan': '1', 'rowspan': '1', 'text': '로얄프라자'}]
]

max_len = max(len(item) for item in group_list)

for idx, items in enumerate(group_list):
    temp = []
    r = [i for i in range(max_len) if not any(int(item['col']) == i for item in items)]
    if len(r) > 0 and idx != 0:
        temp = [group_list[idx-1][row] for row in r ]
    
    new_item = temp + items
    group_list[idx] = new_item

    for z in group_list[idx]:
        print(z)
    print("\n\n")

{'row': '2', 'col': '0', 'colspan': '1', 'rowspan': '4', 'text': '3월1일'}
{'row': '2', 'col': '1', 'colspan': '1', 'rowspan': '1', 'text': '1회'}
{'row': '2', 'col': '2', 'colspan': '1', 'rowspan': '1', 'text': '7:01'}
{'row': '2', 'col': '3', 'colspan': '1', 'rowspan': '1', 'text': '1.6-2.0'}
{'row': '2', 'col': '4', 'colspan': '1', 'rowspan': '1', 'text': '300.0'}
{'row': '2', 'col': '5', 'colspan': '1', 'rowspan': '1', 'text': '7k+771.25~774.25(종점)'}
{'row': '2', 'col': '6', 'colspan': '1', 'rowspan': '1', 'text': 'N/T'}
{'row': '2', 'col': '7', 'colspan': '1', 'rowspan': '1', 'text': 'N/T'}
{'row': '2', 'col': '8', 'colspan': '1', 'rowspan': '1', 'text': 'N/T'}
{'row': '2', 'col': '9', 'colspan': '1', 'rowspan': '1', 'text': '국원물산'}



{'row': '2', 'col': '0', 'colspan': '1', 'rowspan': '4', 'text': '3월1일'}
{'row': '3', 'col': '1', 'colspan': '1', 'rowspan': '2', 'text': '2회'}
{'row': '3', 'col': '2', 'colspan': '1', 'rowspan': '2', 'text': '11:47'}
{'row': '3', 'col': '3', 'colspan'

In [43]:
serialize_list = []
find_word = ['일시', '시간', 'cm', 'dB', '측정위치']

columns = [item for item in columns if int(item['colspan']) <= 1 and any(word in item['text'] for word in find_word)]
for items in group_list:
    data = {}
    for item in items:
        for column in columns:
            if item['col'] == column['col']:
                data[column['text']] = item['text']
    serialize_list.append(data)

serialize_list

[{'일시': '3월1일',
  '시간': '7:01',
  '발파진동(cm/s)': 'N/T',
  '진동레벨dB(V)': 'N/T',
  '소음레벨dB(A)': 'N/T',
  '측정위치': '국원물산'},
 {'일시': '3월1일',
  '시간': '11:47',
  '발파진동(cm/s)': '0.209',
  '진동레벨dB(V)': '72.51',
  '소음레벨dB(A)': '73.35',
  '측정위치': '풍전빌딩'},
 {'일시': '3월1일',
  '시간': '11:47',
  '발파진동(cm/s)': '0.141',
  '진동레벨dB(V)': '71.42',
  '소음레벨dB(A)': '74.79',
  '측정위치': '로얄프라자'},
 {'일시': '3월2일',
  '시간': '7:02',
  '발파진동(cm/s)': '0.082',
  '진동레벨dB(V)': '63.14',
  '소음레벨dB(A)': '60.99',
  '측정위치': '국원물산'},
 {'일시': '3월2일',
  '시간': '11:51',
  '발파진동(cm/s)': '0.074',
  '진동레벨dB(V)': '62.82',
  '소음레벨dB(A)': '66.59',
  '측정위치': '풍전빌딩'},
 {'일시': '3월2일',
  '시간': '11:51',
  '발파진동(cm/s)': '0.051',
  '진동레벨dB(V)': '62.35',
  '소음레벨dB(A)': '67.68',
  '측정위치': '로얄프라자'},
 {'일시': '3월2일',
  '시간': '18:01',
  '발파진동(cm/s)': '0.103',
  '진동레벨dB(V)': '61.62',
  '소음레벨dB(A)': '66.73',
  '측정위치': '국원물산'},
 {'일시': '3월3일',
  '시간': '7:01',
  '발파진동(cm/s)': '0.083',
  '진동레벨dB(V)': '63.22',
  '소음레벨dB(A)': '62.45',
  '측정위치': '국원물산'},
 {'일시':

In [44]:
import pandas as pd

# pandas DataFrame으로 변환
df = pd.DataFrame(serialize_list)

# 계측 위치별로 데이터 프레임 분류
locations = df['측정위치'].unique()  # 계측 위치별 고유값을 찾음
print(df['측정위치'].unique())

# 각 위치별 데이터 프레임을 딕셔너리에 저장
dfs = {}
for location in locations:
    dfs[location] = df[df['측정위치'] == location]

# 결과 확인
# 예시: 과천지식정보타운 위치의 데이터 프레임 출력
print(dfs['국원물산'])

['국원물산' '풍전빌딩' '로얄프라자']
       일시     시간 발파진동(cm/s) 진동레벨dB(V) 소음레벨dB(A)  측정위치
0    3월1일   7:01        N/T       N/T       N/T  국원물산
3    3월2일   7:02      0.082     63.14     60.99  국원물산
6    3월2일  18:01      0.103     61.62     66.73  국원물산
7    3월3일   7:01      0.083     63.22     62.45  국원물산
8    3월4일  16:25        N/T       N/T       N/T  국원물산
9    3월4일  18:01      0.063     57.19     58.42  국원물산
10   3월5일   7:05        N/T       N/T       N/T  국원물산
11   3월5일  17:38      0.117      62.2     60.32  국원물산
12   3월6일   7:01       0.04         -     56.03  국원물산
13   3월6일  17:05       0.06      58.3     55.55  국원물산
14   3월7일   7:00      0.081      58.9     56.14  국원물산
15   3월7일  17:19      0.038     55.68     69.16  국원물산
16   3월8일   7:21      0.037     54.24     54.27  국원물산
17   3월8일  17:36        N/T       N/T       N/T  국원물산
18   3월9일   9:14      0.048     52.73      52.0  국원물산
19   3월9일  17:53       0.04      52.0     58.51  국원물산
20  3월10일   7:01      0.055     55.25     57.77  국원물산
21  